In [ ]:
import json

f = open('parameters.json')

data = json.load(f)

includes = data['include']
excludes = data['exclude']

if len(includes) > 0:
    print("there are includes")
elif len(excludes) > 0:
    print("there are excludes")
else:
    print("There are no parameters given")

In [ ]:
import requests

s = requests.Session()
s.headers.update({"platform":"pc", "language":"en"})

all_items = s.get("https://api.warframe.market/v1/items").json()
items = []

for index, possible_item in enumerate(all_items["payload"]["items"]):
    for string in excludes:
        if string in possible_item["url_name"]:
            all_items["payload"]["items"].pop(index)
        else:
            items.append(possible_item["url_name"])

print(len(all_items["payload"]["items"]))
for i in items:
    print(i)

In [6]:
import statistics
import time
import csv

# write out results to sellPrices.csv
f = open('allOrderPrices.csv', 'w')
writer = csv.writer(f)
headers = ["url_name", "sell_mean", "sell_median", "buy_mean", "buy_median"]
writer.writerow(headers)

print("total items : " + str(len(all_items["payload"]["items"])))

start_time = time.time();

for index, item in enumerate(all_items["payload"]["items"]):

    url_name = item["url_name"]
    print(str(index) + "/" + str(len(all_items["payload"]["items"])) + " " + url_name)
    

    sellRequest = s.get("https://api.warframe.market/v1/items/" + url_name + "/orders").json()
    all_online_sell_prices = []
    all_online_buy_prices = []

    for sItem in sellRequest["payload"]["orders"]:
        user = sItem["user"]
        # ToDo : accept orders for players who have been offline, but for less than 12hr
        if user["status"] == "online" or user["status"] == "ingame":
            if(sItem["order_type"] == "sell"):
                all_online_sell_prices.append(sItem["platinum"])
            else:
                all_online_buy_prices.append(sItem["platinum"])

    # only add to table if item is available
    if len(all_online_sell_prices) > 0:
        sell_mean = statistics.mean(all_online_sell_prices)
        sell_median = statistics.median(all_online_sell_prices)
    if len(all_online_buy_prices) > 0:
        buy_mean = statistics.mean(all_online_buy_prices)
        buy_median = statistics.median(all_online_buy_prices)

    data = [url_name, sell_mean, sell_median, buy_mean, buy_median]
    writer.writerow(data)

    # Warframe.market API has a limit of 3 requests per second
    time.sleep(0.34)

f.close()
print("Total time elapsed: " + str(time.time() - start_time) + "s") 

total items : 2664
0/2664 secura_dual_cestra
1/2664 creeping_bullseye
2/2664 mutalist_alad_v_assassinate_key
3/2664 irradiating_disarm
4/2664 antimatter_absorb
5/2664 arcane_barrier
6/2664 arcane_ice
7/2664 telos_boltor
8/2664 frost_prime_set
9/2664 catalyzer_link
10/2664 targeting_subsystem
11/2664 strun_wraith_receiver
12/2664 tectonic_fracture
13/2664 mantis_set
14/2664 acid_shells
15/2664 nightwatch_napalm
16/2664 quasar_drill
17/2664 comet_blast
18/2664 nebula_bore
19/2664 kaszas_blade
20/2664 velocitus_receiver
21/2664 fluctus_barrel
22/2664 fluctus_limbs
23/2664 fluctus_stock
24/2664 decurion_barrel
25/2664 hydroid_prime_set
26/2664 cleanse_corrupted
27/2664 kappa_beacon
28/2664 star_crimzian
29/2664 virtuos_strike
30/2664 hunter_adrenaline
31/2664 viper_wraith
32/2664 orokin_derelict_plaza_scene
33/2664 grineer_shipyards_manufactory_scene
34/2664 infested_ship_bridge_scene
35/2664 lua_nursery_scene
36/2664 corpus_gas_city_conduit_scene
37/2664 grineer_settlement_reactor_scene
3

In [ ]:
import statistics
import time
import csv

# write out results to buyPrices.csv
f = open('buyPrices.csv', 'w')
writer = csv.writer(f)

print("total items : " + str(len(all_items["payload"]["items"])))

start_time = time.time();

for index, item in enumerate(all_items["payload"]["items"]):

    print(str(index) + "/" + str(len(all_items["payload"]["items"])) + " " + url_name)
    url_name = item["url_name"]

    sellRequest = s.get("https://api.warframe.market/v1/items/" + url_name + "/orders").json()
    all_online_buy_prices = []

    for sItem in sellRequest["payload"]["orders"]:
        user = sItem["user"]
        if(sItem["order_type"] == "buy"):
            # ToDo : accept orders for players who have been offline, but for less than 12hr
            if user["status"] == "online" or user["status"] == "ingame":
                all_online_buy_prices.append(sItem["platinum"])

    # only add to table if item is available
    if len(all_online_buy_prices) > 0:
        mean = statistics.mean(all_online_buy_prices)
        median = statistics.median(all_online_buy_prices)

        data = [url_name, mean, median]
        writer.writerow(data)

    # Warframe.market API has a limit of 3 requests per second
    time.sleep(0.34)

f.close()
print("Total time elapsed: " + str(time.time() - start_time) + "s") 